### MP1 Part2 Solution when using google API

In [1]:
from google import genai
from google_genai_api import GEMINI_API_KEY
import json
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# Read in the dataset and vectorize usinf tf-idf
with open('physics_context.json', 'r') as f:
        data = json.load(f)
client = genai.Client(api_key=GEMINI_API_KEY)

# Prepare the knowledge base for search
knowledge_base = [item['context'] for item in data]
vectorizer = TfidfVectorizer().fit(knowledge_base)
tfidf_matrix = vectorizer.transform(knowledge_base)

In [4]:
# Functions to do retrieval
def find_relevant_context(question):
    """Finds the most relevant context from the knowledge base."""
    # Convert the question to a vector
    question_vector = vectorizer.transform([question])
    # Calculate similarity scores
    similarity_scores = cosine_similarity(question_vector, tfidf_matrix)
    # Get the index of the most similar context
    most_similar_index = similarity_scores.argmax()
    return data[most_similar_index]

def get_response(question):
    # Retrieve the most relevant data point
    relevant_item = find_relevant_context(question)
    
    # The prompt
    augmented_prompt = f"""You are a helpful and knowledgeable physics assistant. 
    Based on the following context, provide a concise and accurate answer to the user's question.
    Do not use any outside knowledge.

    Context: {relevant_item['context']}
    Question: {question}
    Answer:"""

    # Generate the response using Gemini API
    response = client.models.generate_content(model='gemini-1.5-flash',contents=augmented_prompt)
    #response = model.generate_content(augmented_prompt)
    
    return response.text.strip()

In [5]:
# Main chatbot loop

while True:
    user_question = input("Ask a physics question (or type 'quit' to exit): ")
    if user_question.lower() == 'quit':
        break
    
    answer = get_response(user_question)
    print(f"Chatbot: {answer}\n")

Ask a physics question (or type 'quit' to exit):  What is the multiverse theory?


Chatbot: The Multiverse Theory proposes that our universe is one of many universes, each potentially having different physical laws and arising from varying initial conditions or quantum fluctuations.



Ask a physics question (or type 'quit' to exit):  quit
